In [15]:
import os
import tqdm
import json
import spacy
import numpy as np

from typing import List

from transformers import RobertaTokenizer

In [3]:
corpus_path = "/data/SECTOR/"

In [4]:
total_num_sectors = len(os.listdir(corpus_path))

In [7]:
class EnCorpusDataset:
    def __init__(self, corpus_path:str, rank:int, total_num_sectors:int):
        self.corpus_path = corpus_path
        self.rank = rank
        self.total_num_sectors = total_num_sectors
        
        # current sector pointer
        self.cur_sector_id = rank
        
        # extra
        self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        
    @staticmethod
    def read_sector(corpus_path, sector_id:int):
        with open(os.path.join(corpus_path, str(sector_id) + ".jsonl"), "r") as f:
            data = f.readlines()
    
    @staticmethod
    def process_data(data:List[str]):
        example = json.loads(line)
        

In [8]:
with open(os.path.join(corpus_path, str(0) + ".jsonl"), "r") as f:
    data = f.readlines()

In [9]:
def merge_splited_delimiters(sents, delimiter):
    return sents

count = 0

def process_sentences(sentences:List[str], state:int=0) -> List[np.array]:
    global count
    delimiters = [",", "。", "、", "，", "\n", ":", ";", '.']
    
    token_sents = []
    for sent in sentences:
        token_sent = tokenizer.tokenize(sent)
        
        if len(token_sent) > max_seq_length:
            # recursive state machine
            if state == 0:
                sents = sent.split("\n")
                sents = [sent + "\n" for sent in sents[:-1]] + [sents[-1]]
                sub_token_sents = process_sentences(sents, state=1)
                if sub_token_sents:
                    token_sents.extend(sub_token_sents)
            elif state == 1:
                sents = sent.split(".")  
                sents = [sent + "." for sent in sents[:-1]] + [sents[-1]]
                sub_token_sents = process_sentences(sents, state=2)
                if sub_token_sents:
                    token_sents.extend(sub_token_sents)
            elif state == 2:
                sents = sent.split(",")  
                sents = [sent + "," for sent in sents[:-1]] + [sents[-1]]
                sub_token_sents = process_sentences(sents, state=3)
                if sub_token_sents:
                    token_sents.extend(sub_token_sents)
            else:
#                 print(state)
                #print(f"{len(token_sent)} for sub sentences")
#                 print(sent)
                count += 1
                return None
                # breakpoint()
            
        else:
            token_sents.append(token_sent)
            
    return token_sents

In [23]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
max_seq_length = 128

In [ ]:
def process_sentences(sentences:List[str], state:int=0) -> List[np.array]:
    
    token_sents = []
    for sent in sentences:
        token_sent = tokenizer.tokenize(sent)

In [11]:
for line in tqdm.tqdm_notebook(data):
    break

In [18]:
sentences = json.loads(line)["sents"]

In [33]:
new_data = []
flag = False

for line in tqdm.tqdm_notebook(data):
    example = json.loads(line)
    token_sents = process(example["sents"])
    
    new_data.append(token_sents)

In [ ]:
class CorpusDataLoader:
    def __init__(self, corpus_path):
        

In [47]:
len(new_data[100])

21

In [49]:
def __init__(self, corpus_path:str, rank:int, total_num_sectors:int):
    self.corpus_path = corpus_path
    self.rank = rank
    self.total_num_sectors = total_num_sectors

    # current sector pointer
    self.cur_sector_id = rank

    # extra
    self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

215525

In [124]:
class CorpusIterator:
    def __init__(self, processed_docs):
        self.processed_docs = processed_docs
        self.total_num_docs = len(processed_docs)

    def __iter__ (self):
        # shuffle the indices
        indices = np.arange(self.total_num_docs)
        np.random.shuffle(indices)

        for doc_index in indices:
            # randomly sample a document
            doc = self.processed_docs[doc_index]

            for i, segment in enumerate(doc):
                # output if the segment is the start of the document
                yield segment, i==0
                

In [126]:
def sents_to_segments(tokenizer, doc_sentences):

    token_segments = []
    current_seq = []

    for count, sent in enumerate(doc_sentences):
        if count > 0:
            sent = " " + sent

        token_sent = tokenizer.tokenize(sent)

        if len(token_sent) > max_seq_length:
            # append last sequence
            token_segments.append(current_seq)

            for i in range(0, len(token_sent) - max_seq_length, max_seq_length):
                token_segments.append(token_sent[i:i+max_seq_length])

            # assign the current seq the tail of token_sent
            current_seq = token_sent[i+max_seq_length:i+max_seq_length*2]
            continue

        if (len(current_seq) + len(token_sent)) > max_seq_length:
            token_segments.append(current_seq)
            current_seq = token_sent
        else:
            current_seq = current_seq + token_sent
    
    if len(current_seq) > 0:
        token_segments.append(current_seq)
    
    # remove empty segment
    token_segments = [seg for seg in token_segments if seg]
    
    return token_segments

In [127]:
def load_corpus(tokenizer, corpus_path:str, sector_id:int, cache_dir:str = None):
    if cache_dir:
        os.makedirs(cache_dir, exist_ok=True)
        cache_path = os.path.join(cache_dir, f"{sector_id}_cache.pkl")
        
        if os.path.exists(cache_path):
            try:
                processed_docs = torch.load(cache_path)
                return processed_docs
            except:
                print("File Corrupted. Data will be re-processed")
        
    # processing data
    with open(os.path.join(corpus_path, str(sector_id) + ".jsonl"), "r") as f:
        data = f.readlines()
    
    processed_docs = []

    print("Processing Data. Takes about 10 mins")
    
    for line in data:
        example = json.loads(line)
        token_segments = sents_to_segments(tokenizer, example["sents"])

        processed_docs.append(token_segments)    
    
    if cache_dir:
        print("Saving Into Cache")
        torch.save(processed_docs, f"{sector_id}_cache.pkl")
        
    return processed_docs

In [ ]:
class CorpusBatchIterator:
    def __init__(self, corpus_path:str, tokenizer, batch_size:int, rank:int):
    """
    Args:
        corpus_path: directory path to store the corpus sectors
        rank: for distributed learning.
    """ 
        self.tokenizer = tokenize
        self.batch_size = batch_size
        self.current_sector_id = 0
        self.total_num_sectors = len(os.listdir(corpus_path))
        
        # process the data and save it into cache
        
    def __iter__(self):
        while True:
            self.iterators = [CorpusIterator(processed_docs) for i in range(self.batch_size)]
            
            try:
                # TODO: extend it with Ray
                batch = [next(self.iterators[i]) for i in range(self.batch_size)]
                yield batch
            except StopIteration:
                # after the iterator finishes, load the next sector
                processed_docs = self.load_corpus()
                self.iterators = [CorpusIterator(processed_docs) for i in range(self.batch_size)]
                

In [107]:
corpus_gen = iter(CorpusGenerator(new_data))

In [115]:
next(corpus_gen)

(['This',
  'Ġis',
  'Ġparticularly',
  'Ġimportant',
  'Ġwhen',
  'Ġconsidering',
  'Ġthe',
  'Ġtypes',
  'Ġof',
  'Ġfirearms',
  'Ġtypically',
  'Ġinvolved',
  'Ġin',
  'Ġcrime',
  '.',
  '"',
  'Policy',
  'Ġmakers',
  'Ġaren',
  "'t",
  'Ġlooking',
  'Ġat',
  'Ġcurrent',
  'Ġdata',
  ',"',
  'Ġstated',
  'ĠJeff',
  'ĠMonroe',
  ',',
  'ĠPh',
  '.',
  'D',
  '.',
  'Ġand',
  'Ġpresident',
  'Ġof',
  'ĠThink',
  '22',
  'three',
  '.',
  'When',
  'Ġasked',
  'Ġabout',
  'Ġtheir',
  'Ġmost',
  'Ġrecent',
  'Ġtwo',
  'Ġfirearms',
  'Ġpurchases',
  ',',
  'Ġ89',
  '.',
  '7',
  'Ġpercent',
  'Ġof',
  'Ġgun',
  '-',
  'own',
  'ing',
  'Ġsurvey',
  'Ġrespondents',
  'Ġstated',
  'Ġthat',
  'Ġthey',
  'Ġpurchased',
  'Ġtheir',
  'Ġfirearms',
  'Ġfrom',
  'Ġa',
  'Ġdealer',
  'Ġthat',
  'Ġconducted',
  'Ġa',
  'Ġbackground',
  'Ġcheck',
  '.',
  'Ġ"',
  'Sep',
  'ar',
  'ating',
  'Ġthe',
  'Ġdata',
  'Ġby',
  'Ġthe',
  'Ġtype',
  'Ġof',
  'Ġfirearm',
  'Ġpurchased',
  'Ġclar',
  'ifies',

In [99]:
corpus_gen

In [50]:
def get_a_number():
    count = 0
    while True:
        count = count + 1
        yield count
    

In [52]:
m = get_a_number()

In [79]:
next(m)

27

In [24]:
token_sent = [1,2,3,4,5,6,7,8,9,0]

In [27]:
list(range(0, len(token_sent)-3, 3))

[0, 3, 6]